In [3]:
from google.colab import drive
drive.mount('/content/drive')

from __future__ import print_function, division
import os
import time
import copy
import json

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms, models
from torch.optim import lr_scheduler

from sklearn.metrics import cohen_kappa_score, accuracy_score
from tqdm.notebook import tqdm

print(os.listdir("/content/drive/MyDrive/DR_Project/input"))


Mounted at /content/drive
['test.csv', 'sample_submission.csv', 'train.csv', 'test_images', 'train_images']


In [4]:
class CreateDataset(Dataset):
    def __init__(self, df_data, data_dir='../input/', transform=None):
        super().__init__()
        self.df = df_data.values
        self.data_dir = data_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        img_name, label = self.df[index]
        img_path = os.path.join(self.data_dir, img_name + '.png')
        image = cv2.imread(img_path)
        if self.transform:
            image = self.transform(image)
        return image, label


test_csv = pd.read_csv('/content/drive/MyDrive/DR_Project/input/test.csv')
test_path = "/content/drive/MyDrive/DR_Project/input/test_images"

test_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
])

test_csv['diagnosis'] = -1  # Dummy labels for compatibility

test_data = CreateDataset(df_data=test_csv, data_dir=test_path, transform=test_transforms)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)


In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.resnet152(pretrained=False)
num_ftrs = model.fc.in_features

model.fc = nn.Sequential(
    nn.Linear(num_ftrs, 512),
    nn.ReLU(),
    nn.Linear(512, 5),
    nn.LogSoftmax(dim=1)
)

checkpoint_path = '/content/drive/MyDrive/DR_Project/models/classifier.pt'
checkpoint = torch.load(checkpoint_path, map_location=device)

# Load state dict based on checkpoint format
if isinstance(checkpoint, dict):
    if 'model_state_dict' in checkpoint:
        model.load_state_dict(checkpoint['model_state_dict'])
    elif 'state_dict' in checkpoint:
        model.load_state_dict(checkpoint['state_dict'])
    else:
        model.load_state_dict(checkpoint)
else:
    model.load_state_dict(checkpoint)

model.to(device)
model.eval()


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
def predict(testloader):
    preds = []
    with torch.no_grad():
        model.eval()
        for batch_i, (data, target) in enumerate(testloader):
            data = data.to(device)
            output = model(data)
            probs = torch.exp(output)  # Since LogSoftmax was used
            pred_labels = torch.argmax(probs, dim=1)
            preds.extend(pred_labels.cpu().numpy())
    return preds

pred_labels = predict(test_loader)


In [ ]:
sample_sub = pd.read_csv('/content/drive/MyDrive/DR_Project/input/sample_submission.csv')

sample_sub['diagnosis'] = pred_labels

submission_path = '/content/drive/MyDrive/DR_Project/submission.csv'
sample_sub.to_csv(submission_path, index=False)
print(f"Submission file saved to {submission_path}")


Submission file saved to /content/drive/MyDrive/DR_Project/submission.csv
